In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
df = pd.read_csv('Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [42]:
X = df.drop('sales',axis=1)
y=df['sales']

In [43]:
from sklearn.preprocessing import PolynomialFeatures

poly_conv = PolynomialFeatures(degree=3,include_bias=False)
poly_feat = poly_conv.fit_transform(X)
poly_feat.shape

(200, 19)

In [44]:
#Train Test split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(poly_feat,y,test_size=0.25,random_state=101)

In [45]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc=StandardScaler()
sc.fit(X_train)

StandardScaler()

In [46]:
#help(StandardScaler)

In [47]:
sc_X_train = sc.transform(X_train)
sc_X_test = sc.transform(X_test)

### Ridge Regression

In [48]:
from sklearn.linear_model import Ridge

In [49]:
rid = Ridge(alpha=10)

In [50]:
rid.fit(sc_X_train,y_train)

Ridge(alpha=10)

In [51]:
rid_pred = rid.predict(sc_X_test)

In [52]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [53]:
MAE = mean_absolute_error(y_test,rid_pred)
MAE

0.5621969787129825

In [54]:
MSE = mean_squared_error(y_test,rid_pred)
MSE

0.5832130131233249

In [55]:
np.sqrt(MSE)

0.7636838436966732

### Ridge Regression CV

In [56]:
from sklearn.linear_model import RidgeCV

rid_cv_model = RidgeCV(alphas=(0.1,0.09,.15,.08,.2))

In [57]:
rid_cv_model.fit(sc_X_train,y_train)

RidgeCV(alphas=(0.1, 0.09, 0.15, 0.08, 0.2))

In [58]:
rid_cv_model.alpha_

0.08

In [59]:
ridCV_pred = rid_cv_model.predict(sc_X_test)

In [60]:
MAECV = mean_absolute_error(y_test,ridCV_pred)
MAECV

0.37677926198628936

In [61]:
MSECV = mean_squared_error(y_test,ridCV_pred)
MSECV

0.28645030347334666

In [62]:
np.sqrt(MSECV)

0.5352105225734511

In [63]:
rid_cv_model.coef_

array([ 6.25004724,  0.41019117,  0.62420531, -7.61217107,  4.61517697,
       -1.59665867, -0.56923988,  0.19254299,  0.12640331,  3.2855302 ,
       -1.23101839,  0.98683981,  0.3437116 ,  0.06516885,  0.10849859,
        0.35445615, -0.28154919, -0.03245422, -0.16495013])

### LASSO Regression

Least Absolute Shrinkage and Selection Operator

In [64]:
from sklearn.linear_model import LassoCV

#help(LassoCV)

In [65]:
lassoCV_m = LassoCV(eps=0.1,n_alphas=100, max_iter=1000000, cv=5)

In [66]:
lassoCV_m.fit(sc_X_train,y_train)

LassoCV(cv=5, eps=0.1, max_iter=1000000)

In [67]:
lassoCV_m.alpha_

0.4955891288263106

In [68]:
lassoCV_m_pred = lassoCV_m.predict(sc_X_test)

In [69]:
MAE_lasso_CV = mean_absolute_error(y_test,lassoCV_m_pred)
MAE_lasso_CV

0.6241496332838827

In [70]:
MSE_lasso_CV = mean_squared_error(y_test,lassoCV_m_pred)
MSE_lasso_CV

0.9355589456162755

In [71]:
np.sqrt(MSE_lasso_CV)

0.967242961006321

In [72]:
lassoCV_m.coef_

array([1.03575888, 0.        , 0.        , 0.        , 3.79481506,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

### Elastic Net

In [73]:
from sklearn.linear_model import ElasticNetCV

e_model = ElasticNetCV(l1_ratio=[.01,.1,0.5,0.9,1],eps=0.1,max_iter=100000,cv=5)

In [74]:
e_model.fit(sc_X_train,y_train)

ElasticNetCV(cv=5, eps=0.1, l1_ratio=[0.01, 0.1, 0.5, 0.9, 1], max_iter=100000)

In [75]:
en_pred = e_model.predict(sc_X_test)

In [76]:
MAE_en = mean_absolute_error(y_test,en_pred)
MAE_en

0.6241496332838827

In [77]:
MSE_en = mean_squared_error(y_test,en_pred)
MSE_en

0.9355589456162755

In [78]:
np.sqrt(MSE_en)

0.967242961006321

In [79]:
e_model.l1_ratio_

1.0

In [80]:
e_model.coef_

array([1.03575888, 0.        , 0.        , 0.        , 3.79481506,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Grid Search

In [81]:
param_grid = {'alpha':[0.1,1,5,10,50,100],
             'l1_ratio':[0.1,0.5,0.7,0.9,0.99,1]}

In [82]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

In [83]:
base_e_model = ElasticNet()

In [85]:
grid_model = GridSearchCV(estimator=base_e_model,
            param_grid=param_grid,
            cv=5)

In [89]:
grid_model.fit(sc_X_train,y_train)

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.99, 1]})

In [90]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=0.99)

In [91]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 0.99}

In [95]:
y_pred = grid_model.predict(sc_X_test)

In [96]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

0.5292083999395513

In [97]:
np.sqrt(mean_squared_error(y_test,y_pred))

0.7274671126171625